In [1]:
pip install torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import re

In [3]:
print(torch.cuda.is_available())

True


In [4]:
# Read csv
file_path = '/content/drive/My Drive/type_of_hate.csv'
df = pd.read_csv(file_path)


In [5]:
df.head()

,cleaned_comment,class
0,You should know women's sports are a joke,2
1,You look like sloth with deeper downs syndrome,1
2,You look like Russian and speak like Indian bo...,7
3,Women deserve to be abused I guess,2
4,Women are made for making babies and cooking d...,2


In [6]:
df['class'].value_counts()

,count
class,
7,82
0,61
5,58
4,58
2,48
6,45
3,44
1,37


In [7]:
df.shape

(433, 2)

In [8]:
# peagsus paraphraser generation function
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

In [9]:
new_comments = []
new_classes = []

# Loop through each row of the dataframe
for index, row in df.iterrows():
    print(index)
    comment = row['cleaned_comment']
    class_label = row['class']

    # Ensure the comment is a valid string
    if isinstance(comment, str) and comment.strip():
        # Generate 10 paraphrases for each comment
        paraphrases = get_response(comment, num_return_sequences=3, num_beams=10)

        # Add the generated paraphrases to the new lists along with the corresponding class
        for paraphrase in paraphrases:
            new_comments.append(paraphrase)
            new_classes.append(class_label)
    else:
        # If the comment is invalid or empty, you may want to handle this case
        print(f"Invalid comment at index {index}: {comment}")

# Create a new DataFrame with the generated paraphrases and their corresponding classes
new_df = pd.DataFrame({
    'cleaned_comment': new_comments,
    'class': new_classes
})




0


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [10]:
new_df2=new_df.copy()

In [11]:
#Generated dataset
new_df.head(3)

,cleaned_comment,class
0,Women's sports are a joke.,2
1,Women's sports are not real.,2
2,Women's sports are just a joke.,2


In [12]:
# df = df.drop(columns=["comment"])


In [13]:
# You can now merge this new DataFrame with the original one or save it
df_fin = pd.concat([df, new_df], ignore_index=True)


In [14]:
df_fin.shape

(1732, 2)

In [15]:
df_fin.tail(15)

,cleaned_comment,class
1717,All people are fucked up because they know how...,2
1718,All people are fucked up because they know how...,2
1719,All people are fucked up because they know how...,2
1720,His mother should have known that he was like ...,0
1721,His mother should have known that he is like t...,0
1722,The weird people his mother should have known ...,0
1723,I am an Arab priest and they sound like jokes.,5
1724,"They sound like they are jokes, but I am an Ar...",5
1725,I am an Arab priest and they sound like they a...,5
1726,You want to fuck your brother instead of your ...,6


In [16]:
df_fin['class'].value_counts()

,count
class,
7,328
0,244
5,232
4,232
2,192
6,180
3,176
1,148


In [18]:
df_fin = df_fin.drop(columns=["comment"])

In [ ]:
df_fin['class'].value_counts()

,count
class,
0,2260
1,1432


In [19]:
df_fin.head(2)

,cleaned_comment,class
0,You should know women's sports are a joke,2
1,You look like sloth with deeper downs syndrome,1


In [20]:
df_fin.to_csv("type_of_hate_fin.csv", index=False)